In [10]:
#### Pureba A/B

### Instrucciones para completar la tarea

1. **Describe los objetivos del estudio.**
   - Explica cuál es el propósito del análisis y qué se espera lograr con la prueba A/B.

2. **Explora los datos:**
   - ¿Es necesario convertir los tipos de datos?
   - ¿Hay valores ausentes o duplicados? Si es así, ¿cómo los caracterizarías?

3. **Lleva a cabo el análisis exploratorio de datos (EDA):**
   - Estudia la conversión en las diferentes etapas del embudo.
   - ¿El número de eventos por usuario está distribuido equitativamente entre las muestras?
   - ¿Hay usuarios que están presentes en ambas muestras?
   - ¿Cómo se distribuye el número de eventos entre los días?
   - ¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?

4. **Evaluar los resultados de la prueba A/B:**
   - ¿Qué puedes decir sobre los resultados de la prueba A/B?
   - Utiliza una prueba z para comprobar la diferencia estadística entre las proporciones.

5. **Describe tus conclusiones:**
   - Resume los hallazgos del análisis exploratorio de datos (EDA).
   - Interpreta los resultados de la prueba A/B y proporciona recomendaciones basadas en los datos.

1. **Describe los objetivos del estudio.**
   - Explica cuál es el propósito del análisis y qué se espera lograr con la prueba A/B.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



def looks(datos):
    """Genera un análisis de todo el set de datos de manera global"""

    print("Se ejecuta la descripción de los datos:")
    print(datos.describe(include="all"))
    print()
    print("se hace exploración con método info:")
    print(datos.info())
    print()
    print("se hace una observación con el método Sample:")
    if len(datos) > 10:
        print(datos.sample(10))
    else:
        print(datos)
    print()
    print("hay NA?:")
    print(datos.isna().sum())
    print()
    print("Duplicados:")
    print(datos.duplicated().sum())

In [12]:
marketing_events = pd.read_csv('datasets/ab_project_marketing_events_us.csv')
looks(marketing_events)

Se ejecuta la descripción de los datos:
                            name regions    start_dt   finish_dt
count                         14      14          14          14
unique                        14       6          14          14
top     Christmas&New Year Promo    APAC  2020-12-25  2021-01-03
freq                           1       4           1           1

se hace exploración con método info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes
None

se hace una observación con el método Sample:
                                name                   regions    start_dt  \
6             Chinese New Year Promo                      APAC  2020-01-25   
4       

In [13]:
new_users = pd.read_csv('datasets/final_ab_new_users_upd_us.csv')
looks(new_users)

Se ejecuta la descripción de los datos:
                 user_id  first_date region   device
count              58703       58703  58703    58703
unique             58703          17      4        4
top     8F04273BB2860229  2020-12-21     EU  Android
freq                   1        6077  43396    26159

se hace exploración con método info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB
None

se hace una observación con el método Sample:
                user_id  first_date     region   device
37614  98715E220A52EA09  2020-12-17         EU       PC
11043  13B427F12098CEF2  2020-12-21         EU      Mac
50680  82903850CE2D64DA  2020-12-13         EU   

In [14]:
events = pd.read_csv('datasets/final_ab_events_upd_us.csv')
looks(events)

Se ejecuta la descripción de los datos:
                 user_id             event_dt event_name       details
count             423761               423761     423761  60314.000000
unique             58703               257138          4           NaN
top     A3917F81482141F2  2020-12-14 18:54:55      login           NaN
freq                  36                   10     182465           NaN
mean                 NaN                  NaN        NaN     23.881219
std                  NaN                  NaN        NaN     72.228884
min                  NaN                  NaN        NaN      4.990000
25%                  NaN                  NaN        NaN      4.990000
50%                  NaN                  NaN        NaN      4.990000
75%                  NaN                  NaN        NaN      9.990000
max                  NaN                  NaN        NaN    499.990000

se hace exploración con método info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to

In [15]:
participants = pd.read_csv('datasets/final_ab_participants_upd_us.csv')
looks(participants)

Se ejecuta la descripción de los datos:
                 user_id  group            ab_test
count              14525  14525              14525
unique             13638      2                  2
top     FABB6D49B11E673F      A  interface_eu_test
freq                   2   8214              10850

se hace exploración con método info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB
None

se hace una observación con el método Sample:
                user_id group                  ab_test
14086  18BAF1836D9FC913     B        interface_eu_test
6402   4FD9BF4AD933388C     B        interface_eu_test
4219   F56DD804E0DA1FFD     A        interface_eu_test
1267   971F376FA64BA110     A  recommender_system_test
1874 

### No se encuentras particulatidades especiales para los datos 

In [16]:
#Se convierten a fechas las columnas correspondientes
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
events['event_dt'] = pd.to_datetime(events['event_dt'], errors='coerce')
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

In [17]:

# Calcular la conversión en las diferentes etapas del embudo
funnel = events.groupby('event_name').size().reset_index(name='count')
funnel['percentage'] = (funnel['count'] / funnel['count'].sum()) * 100
print("Conversión en las diferentes etapas del embudo:")
print(funnel)

# Crear el DataFrame events_with_groups combinando events y participants
events_with_groups = events.merge(participants, on='user_id', how='inner')

# Número de eventos por usuario en cada muestra
events_per_user = events_with_groups.groupby(['user_id', 'group']).size().reset_index(name='event_count')
print("\nDistribución del número de eventos por usuario en cada muestra:")
print(events_per_user.groupby('group')['event_count'].describe())

# Verificar si hay usuarios presentes en ambas muestras
users_in_both_groups = participants.groupby('user_id')['group'].nunique().reset_index()
users_in_both_groups = users_in_both_groups[users_in_both_groups['group'] > 1]
print(f"\nNúmero de usuarios presentes en ambas muestras: {len(users_in_both_groups)}")

# Distribución del número de eventos entre los días
events['event_date'] = events['event_dt'].dt.date
events_per_day = events.groupby('event_date').size()
print("\nDistribución del número de eventos entre los días:")
print(events_per_day)

# Peculiaridades en los datos
print("\nPeculiaridades en los datos:")
print("Eventos con valores nulos:")
print(events.isna().sum())
print("\nUsuarios duplicados en participants:")
print(participants.duplicated(subset='user_id').sum())

Conversión en las diferentes etapas del embudo:
     event_name   count  percentage
0         login  182465   43.058469
1  product_cart   60120   14.187242
2  product_page  120862   28.521266
3      purchase   60314   14.233023

Distribución del número de eventos por usuario en cada muestra:
        count     mean       std  min  25%  50%  75%   max
group                                                     
A      7874.0  7.45555  4.475159  1.0  4.0  6.0  9.0  40.0
B      6205.0  7.11249  4.246201  1.0  4.0  6.0  9.0  36.0

Número de usuarios presentes en ambas muestras: 441

Distribución del número de eventos entre los días:
event_date
2020-12-07    11385
2020-12-08    12547
2020-12-09    12122
2020-12-10    14077
2020-12-11    13864
2020-12-12    17634
2020-12-13    20985
2020-12-14    26184
2020-12-15    23469
2020-12-16    20909
2020-12-17    21751
2020-12-18    22871
2020-12-19    24273
2020-12-20    26425
2020-12-21    32559
2020-12-22    29472
2020-12-23    26108
2020-12-24    1

### Se encuentra que existen pocos eventos con valores nulos, y que hay ID duplicados esto producto que cada usuario tiene mas de un evento.

In [19]:
from statsmodels.stats.proportion import proportions_ztest

# Calcular conversiones por grupo
conversiones = events_with_groups.groupby('group')['user_id'].nunique()
usuarios_totales = participants.groupby('group')['user_id'].nunique()

# Conversiones y usuarios totales por grupo
conversiones_A = conversiones['A']
conversiones_B = conversiones['B']
usuarios_A = usuarios_totales['A']
usuarios_B = usuarios_totales['B']

# Proporciones de conversión
proporcion_A = conversiones_A / usuarios_A
proporcion_B = conversiones_B / usuarios_B

print(f"Proporción de conversión en el grupo A: {proporcion_A:.2%}")
print(f"Proporción de conversión en el grupo B: {proporcion_B:.2%}")

# Prueba z para comparar proporciones
conversiones = [conversiones_A, conversiones_B]
usuarios = [usuarios_A, usuarios_B]
stat, p_value = proportions_ztest(conversiones, usuarios)

print("\nResultados de la prueba z:")
print(f"Estadístico z: {stat:.2f}")
print(f"Valor p: {p_value:.4f}")

# Conclusiones
if p_value < 0.05:
    print("\nConclusión: Hay una diferencia estadísticamente significativa entre las proporciones de conversión de los grupos A y B.")
else:
    print("\nConclusión: No hay una diferencia estadísticamente significativa entre las proporciones de conversión de los grupos A y B.")

Proporción de conversión en el grupo A: 100.00%
Proporción de conversión en el grupo B: 100.00%

Resultados de la prueba z:
Estadístico z: nan
Valor p: nan

Conclusión: No hay una diferencia estadísticamente significativa entre las proporciones de conversión de los grupos A y B.


c:\Users\Branchbit Reclu\miniconda3\envs\TT\Lib\site-packages\statsmodels\stats\weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std


### Al no existir una diferencia entre los grupos, se sugiere replantear la estrategia de marketing empleada y buscar alternativas que puedan tener mayor efecto en los resultados 